<a href="https://colab.research.google.com/github/vindhya-jain/CanvasCart/blob/main/art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faker

# START

In [ ]:
import sqlite3
from faker import Faker
import random
import string


In [ ]:
# Initialize Faker for generating dummy data
fake = Faker()

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('CanvasCart.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()


In [ ]:
# Drop the Artwork table
cursor.execute('DROP TABLE IF EXISTS Artwork')

### Create Table





In [ ]:
# Create User table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS User (
        UserID INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        Email TEXT NOT NULL UNIQUE,
        Role TEXT CHECK(Role IN ('Artist', 'Buyer')) NOT NULL
    )
''')
# Create Authentication  table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS AuthenticationSystem (
        LoginID INTEGER PRIMARY KEY,
        Password VARCHAR NOT NULL,
        FOREIGN KEY (LoginID) REFERENCES User(UserID)

    )
''')

# Create Artist table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Artist (
        ArtistID INTEGER PRIMARY KEY,
        Biography TEXT,
        PortfolioURL TEXT,
        FOREIGN KEY (ArtistID) REFERENCES User(UserID)
    )
''')

# Create Buyer table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Buyer (
        BuyerID INTEGER PRIMARY KEY,
        ShippingAddress TEXT NOT NULL,
        Membership INTEGER NOT NULL CHECK(Membership IN (0, 1)),
        FOREIGN KEY (BuyerID) REFERENCES User(UserID)
    )
''')

# Create Artwork table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Artwork (
        ArtworkID TEXT PRIMARY KEY,
        ArtistID INTEGER,
        Title TEXT NOT NULL,
        Description TEXT,
        ImageURL TEXT,
        Medium TEXT,
        Style TEXT,
        Price REAL NOT NULL,
        Availability INTEGER NOT NULL CHECK(Availability >= 0),
        FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID)
    )
''')

# Create Order table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS "Orders" (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        BuyerID INTEGER,
        OrderDate DATE DEFAULT CURRENT_DATE,
        TotalAmount REAL NOT NULL,
        PaymentStatus TEXT CHECK(PaymentStatus IN ('Pending', 'Completed', 'Failed')) NOT NULL,
        ShippingStatus TEXT CHECK(ShippingStatus IN ('Order Confirmed', 'Shipped', 'Delivered', 'Canceled')) NOT NULL,
        FOREIGN KEY (BuyerID) REFERENCES Buyer(BuyerID)
    )
''')

# Create Item table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Item (
        ArtworkID INTEGER,
        OrderID INTEGER,
        Quantity INTEGER NOT NULL,
        Amount REAL,
        PRIMARY KEY (ArtworkID, OrderID),
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ArtworkID) REFERENCES Artwork(ArtworkID)
    )
''')

# Create a trigger to update Amount when a new row is inserted or updated
cursor.execute('''
    CREATE TRIGGER IF NOT EXISTS update_item_amount
    AFTER INSERT ON Item
    FOR EACH ROW
    BEGIN
        UPDATE Item
        SET Amount = (
            SELECT Price FROM Artwork WHERE ArtworkID = NEW.ArtworkID
        ) * NEW.Quantity
        WHERE ArtworkID = NEW.ArtworkID;
    END
''')

# Create ShoppingCart table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ShoppingCart (
        CartID INTEGER PRIMARY KEY,
        LastUpdatedDate DATE,
        LastUpdatedTime TIME,
        FOREIGN KEY (CartID) REFERENCES Buyer(BuyerID)
    )
''')

# Create Payment table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Payment (
        PaymentID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID INTEGER,
        PaymentMethod TEXT CHECK ( PaymentMethod IN ('UPI', 'CreditCard', 'DebitCard', 'NetBanking','COD')) NOT NULL,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID)
    )
''')

# Create Review table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Review (
        ReviewID INTEGER PRIMARY KEY AUTOINCREMENT,
        BuyerID INTEGER,
        ArtworkID INTEGER,
        Rating INTEGER NOT NULL CHECK(Rating >= 1 AND Rating <= 5),
        ReviewText TEXT NOT NULL,
        FOREIGN KEY (BuyerID) REFERENCES Buyer(BuyerID),
        FOREIGN KEY (ArtworkID) REFERENCES Artwork(ArtworkID)
    )
''')



### Extra

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# List of all tables to be emptied
tables = [
    'Review',
    'Payment',
    'Item',
    'Orders',
    'ShoppingCart',
    'Artwork',
    'Buyer',
    'Artist',
    'AuthenticationSystem',
    'User'
]

# Disable foreign key checks to avoid constraint errors while truncating
cursor.execute('PRAGMA foreign_keys = OFF')

# Empty each table
for table in tables:
    cursor.execute(f'DELETE FROM {table}')
    cursor.execute(f'DELETE FROM sqlite_sequence WHERE name="{table}"')  # Reset autoincrement counter

# Re-enable foreign key checks
cursor.execute('PRAGMA foreign_keys = ON')

# Commit the changes
conn.commit()

print("All tables have been emptied.")


All tables have been emptied.


In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Query to show all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch and print the table names
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])


Tables in the database:
User
sqlite_sequence
AuthenticationSystem
Artist
Buyer
Orders
Item
ShoppingCart
Payment
Review
Artwork


In [ ]:
# Execute a query to retrieve all rows from the User table
cursor.execute('SELECT * FROM User')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print each row
for row in rows:
    print(row)



### Insert dummy data

In [ ]:
from faker import Faker
import sqlite3
import random

# Initialize Faker for generating dummy data
fake = Faker()

# Connect to SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Function to insert data into the User, AuthenticationSystem, Artist, and Buyer tables
def insert_users_and_related_tables(num_artists, num_buyers):
    artist_ids = []
    buyer_ids = []

    # Insert artists
    for _ in range(num_artists):
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = fake.email()
        cursor.execute('''
            INSERT INTO User (FirstName, LastName, Email, Role)
            VALUES (?, ?, ?, ?)
        ''', (first_name, last_name, email, 'Artist'))
        artist_id = cursor.lastrowid
        artist_ids.append(artist_id)

        # Insert into AuthenticationSystem
        cursor.execute('''
            INSERT INTO AuthenticationSystem (LoginID, Password)
            VALUES (?, ?)
        ''', (artist_id, fake.password()))

        # Insert into Artist table
        cursor.execute('''
            INSERT INTO Artist (ArtistID, Biography, PortfolioURL)
            VALUES (?, ?, ?)
        ''', (artist_id, fake.text(), fake.url()))

    # Insert buyers
    for _ in range(num_buyers):
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = fake.email()
        cursor.execute('''
            INSERT INTO User (FirstName, LastName, Email, Role)
            VALUES (?, ?, ?, ?)
        ''', (first_name, last_name, email, 'Buyer'))
        buyer_id = cursor.lastrowid
        buyer_ids.append(buyer_id)

        # Insert into AuthenticationSystem
        cursor.execute('''
            INSERT INTO AuthenticationSystem (LoginID, Password)
            VALUES (?, ?)
        ''', (buyer_id, fake.password()))

        # Insert into Buyer table
        cursor.execute('''
            INSERT INTO Buyer (BuyerID, ShippingAddress, Membership)
            VALUES (?, ?, ?)
        ''', (buyer_id, fake.address(), random.choice([0, 1])))

    # Commit the changes
    conn.commit()
    return artist_ids, buyer_ids

# Function to insert data into the Artwork, Order, Item, ShoppingCart, Payment, and Review tables
def insert_artworks_and_related_tables(artist_ids, buyer_ids):
    artwork_ids = []
    medium_counts = {}  # Dictionary to track counts for each medium

    # Insert artworks
    for artist_id in artist_ids:
        for _ in range(random.randint(0, 5)):  # Each artist creates 1-5 artworks
            medium = random.choice(['Oil', 'Acrylic', 'Watercolor', 'Digital','Sculpture'])
            medium_prefix = medium[:3].upper()
            if medium_prefix not in medium_counts:
                medium_counts[medium_prefix] = 0
            medium_counts[medium_prefix] += 1
            artwork_id = f'{medium_prefix}_{str(medium_counts[medium_prefix]).zfill(2)}'

            cursor.execute('''
                INSERT INTO Artwork (ArtworkID, ArtistID, Title, Description, ImageURL, Medium, Style, Price, Availability)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                artwork_id,
                artist_id,
                fake.sentence(nb_words=3),
                fake.text(),
                fake.url(),
                medium,
                random.choice(['Abstract', 'Realism', 'Impressionism', 'Modern']),
                round(random.uniform(100.0, 10000.0), 2),
                random.randint(1, 20)  # Number of units available
            ))
            artwork_ids.append(artwork_id)

    # Insert orders and related data
    for buyer_id in buyer_ids:
        for _ in range(random.randint(0, 3)):  # Each buyer places 1-3 orders
            cursor.execute('''
                INSERT INTO "Orders" (BuyerID, TotalAmount, PaymentStatus, ShippingStatus)
                VALUES (?, ?, ?, ?)
            ''', (
                buyer_id,
                round(random.uniform(100.0, 5000.0), 2),
                random.choice(['Pending', 'Completed', 'Failed']),
                random.choice(['Order Confirmed', 'Shipped', 'Delivered', 'Canceled'])
            ))
            order_id = cursor.lastrowid

            # Insert items related to the order
            selected_artworks = set()  # Track selected artworks for each order
            for _ in range(random.randint(1, 5)):  # Each order has 1-5 items
                while True:  # Ensure no duplicate ArtworkID in the same order
                    artwork_id = random.choice(artwork_ids)
                    if artwork_id not in selected_artworks:
                        selected_artworks.add(artwork_id)
                        break
                quantity = random.randint(1, 5)

                # Fetch the price of the selected artwork
                cursor.execute('''
                    SELECT Price FROM Artwork WHERE ArtworkID = ?
                ''', (artwork_id,))
                price = cursor.fetchone()[0]

                # Check if this ArtworkID already exists in the current order
                cursor.execute('''
                    SELECT Quantity FROM Item WHERE OrderID = ? AND ArtworkID = ?
                ''', (order_id, artwork_id))
                existing_item = cursor.fetchone()

                if existing_item:
                    # If it exists, increment the quantity
                    new_quantity = existing_item[0] + quantity
                    cursor.execute('''
                        UPDATE Item SET Quantity = ?, Amount = ? * Quantity WHERE OrderID = ? AND ArtworkID = ?
                    ''', (new_quantity, price, order_id, artwork_id))
                else:
                    # If it doesn't exist, insert a new row
                    amount = price * quantity
                    cursor.execute('''
                        INSERT INTO Item (OrderID, ArtworkID, Quantity, Amount)
                        VALUES (?, ?, ?, ?)
                    ''', (order_id, artwork_id, quantity, amount))

            # Insert into ShoppingCart
            cursor.execute('''
                SELECT CartID FROM ShoppingCart WHERE CartID = ?
            ''', (buyer_id,))
            cart_exists = cursor.fetchone()

            if not cart_exists:  # If no cart exists, insert a new one
                cursor.execute('''
                    INSERT INTO ShoppingCart (CartID, LastUpdatedDate, LastUpdatedTime)
                    VALUES (?, ?, ?)
                ''', (
                    buyer_id,
                    fake.date(),
                    fake.time()
                ))
            else:  # If cart exists, update it
                cursor.execute('''
                    UPDATE ShoppingCart
                    SET LastUpdatedDate = ?, LastUpdatedTime = ?
                    WHERE CartID = ?
                ''', (
                    fake.date(),
                    fake.time(),
                    buyer_id
                ))

            # Insert payment details
            cursor.execute('''
                INSERT INTO Payment (OrderID, PaymentMethod)
                VALUES (?, ?)
            ''', (order_id, random.choice(['UPI', 'CreditCard', 'DebitCard', 'NetBanking', 'COD'])))

            # Insert reviews
            for _ in range(random.randint(1, 2)):  # Each buyer reviews 1-2 items
                artwork_id = random.choice(artwork_ids)
                cursor.execute('''
                    INSERT INTO Review (BuyerID, ArtworkID, Rating, ReviewText)
                    VALUES (?, ?, ?, ?)
                ''', (
                    buyer_id,
                    artwork_id,
                    random.randint(1, 5),
                    fake.paragraph()
                ))

    # Commit the changes
    conn.commit()

# Populate the database with dummy data
artist_ids, buyer_ids = insert_users_and_related_tables(10, 40)
insert_artworks_and_related_tables(artist_ids, buyer_ids)

# Retrieve and print some data to verify
cursor.execute('SELECT * FROM User')
users = cursor.fetchall()
print("\nUsers:")
for user in users:
    print(user)

cursor.execute('SELECT * FROM Artwork')
artworks = cursor.fetchall()
print("\nArtworks:")
for artwork in artworks:
    print(artwork)

cursor.execute('SELECT * FROM Item ORDER BY OrderID')
items = cursor.fetchall()
print("\nItems:")
for item in items:
    print(item)




Users:
(1, 'Mark', 'Smith', 'alison69@example.net', 'Artist')
(2, 'Taylor', 'Andrews', 'rodriguezmegan@example.com', 'Artist')
(3, 'Holly', 'Alvarado', 'valentinesean@example.net', 'Artist')
(4, 'Jeffrey', 'Martinez', 'woodalejandro@example.com', 'Artist')
(5, 'Sara', 'Newman', 'abright@example.net', 'Artist')
(6, 'Brett', 'Clements', 'elizabethshaw@example.net', 'Artist')
(7, 'Ryan', 'Harrison', 'hbridges@example.org', 'Artist')
(8, 'Suzanne', 'Wright', 'qhowell@example.net', 'Artist')
(9, 'John', 'Martinez', 'belindagilbert@example.net', 'Artist')
(10, 'Candice', 'Park', 'sydney78@example.com', 'Artist')
(11, 'Melissa', 'Yoder', 'michael77@example.org', 'Buyer')
(12, 'Kristina', 'Walsh', 'hickschristopher@example.org', 'Buyer')
(13, 'Jessica', 'Robinson', 'christopherbenjamin@example.com', 'Buyer')
(14, 'Mark', 'Wu', 'nguyensharon@example.com', 'Buyer')
(15, 'Eugene', 'Rhodes', 'bonniesimon@example.com', 'Buyer')
(16, 'Regina', 'Sampson', 'daniel02@example.com', 'Buyer')
(17, 'Robin

In [ ]:
cursor.execute('SELECT * FROM ARTIST')
artists = cursor.fetchall()
print("\nArtist:")
for artist in artists:
    print(artist)


Artist:
(1, 'Partner expert discover hold appear. Value offer practice want. Least forget suffer final call PM leave writer.', 'https://www.chung.com/')
(2, 'Them add note you though environment though. Son record already many concern. Little surface positive thus carry look know. Only sea position.', 'https://mitchell.info/')
(3, 'Quality mind deep through. Law mean term spring something.\nMedia it yard cell. Newspaper party and service data never example. Treatment kitchen study include report behavior.', 'https://www.hall-bentley.biz/')
(4, 'Mind item likely clear billion. Produce whose business despite their red.\nMan see military course. One when real people. Growth focus rule discussion at quite.', 'http://www.house.org/')
(5, 'Mrs paper art first grow. Accept tax person quickly activity subject meet.\nThere fight down sense against five.\nView main sea whose. Camera task put any arm tough.\nUnderstand yourself customer call.', 'http://wagner-jordan.info/')
(6, 'Dark like networ

In [ ]:
# import sqlite3

# # Connect to SQLite database
# conn = sqlite3.connect('CanvasCart.db')
# cursor = conn.cursor()

# # Add OrderTime column to the Orders table
# cursor.execute('''
#     ALTER TABLE Orders
#     ADD COLUMN OrderTime TIME
# ''')

# conn.commit()


In [ ]:
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Function to generate a random time between 1 PM and 11 PM
def generate_random_time():
    hour = random.randint(13, 22)  # 1 PM is 13:00, 11 PM is 22:00
    minute = random.randint(0, 59)
    return f"{hour:02}:{minute:02}:00"

# Function to generate a random date within a specified range
def generate_random_date(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    random_date = start + timedelta(days=random.randint(0, (end - start).days))
    return random_date.strftime('%Y-%m-%d')

# Update OrderDate and OrderTime for existing orders
cursor.execute('SELECT OrderID FROM Orders')
orders = cursor.fetchall()

start_date = '2021-01-01'
end_date = '2024-09-01'

for (order_id,) in orders:
    random_date = generate_random_date(start_date, end_date)
    random_time = generate_random_time()
    cursor.execute('''
        UPDATE Orders
        SET OrderDate = ?, OrderTime = ?
        WHERE OrderID = ?
    ''', (random_date, random_time, order_id))

conn.commit()


In [ ]:
cursor.execute('SELECT * FROM Orders')
orders = cursor.fetchall()
print("\nOrders:")
for order in orders:
    print(order)


Orders:
(1, 11, '2024-03-20', 1692.68, 'Failed', 'Order Confirmed', '15:35:00')
(2, 11, '2023-10-18', 3162.51, 'Failed', 'Canceled', '14:22:00')
(3, 11, '2023-05-12', 272.2, 'Failed', 'Order Confirmed', '21:18:00')
(4, 12, '2023-03-13', 4965.8, 'Completed', 'Canceled', '19:30:00')
(5, 13, '2023-05-03', 4014.39, 'Completed', 'Canceled', '14:33:00')
(6, 13, '2023-11-22', 3225.23, 'Completed', 'Order Confirmed', '19:05:00')
(7, 13, '2022-09-17', 2838.7, 'Completed', 'Delivered', '18:05:00')
(8, 15, '2022-08-01', 4904.53, 'Failed', 'Order Confirmed', '13:29:00')
(9, 16, '2021-07-07', 2948.39, 'Completed', 'Shipped', '20:27:00')
(10, 17, '2024-07-06', 2766.17, 'Pending', 'Shipped', '18:23:00')
(11, 18, '2021-03-02', 2613.22, 'Failed', 'Shipped', '13:08:00')
(12, 18, '2022-03-01', 717.59, 'Completed', 'Canceled', '15:51:00')
(13, 18, '2023-01-08', 1051.79, 'Failed', 'Order Confirmed', '21:36:00')
(14, 19, '2024-01-14', 4893.27, 'Failed', 'Delivered', '14:56:00')
(15, 19, '2023-12-17', 1653.

### Display tables

In [ ]:
import pandas as pd
def display_table(query):
    cursor.execute(query)
    columns = [description[0] for description in cursor.description]
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=columns)
    print(df.to_string(index=False))

# Display tables without specifying columns
display_table('SELECT * FROM User')
display_table('SELECT * FROM Artwork')
display_table('SELECT * FROM Orders')
display_table('SELECT * FROM Item ORDER BY OrderID')


 UserID  FirstName   LastName                           Email   Role
      1       Mark      Smith            alison69@example.net Artist
      2     Taylor    Andrews      rodriguezmegan@example.com Artist
      3      Holly   Alvarado       valentinesean@example.net Artist
      4    Jeffrey   Martinez       woodalejandro@example.com Artist
      5       Sara     Newman             abright@example.net Artist
      6      Brett   Clements       elizabethshaw@example.net Artist
      7       Ryan   Harrison            hbridges@example.org Artist
      8    Suzanne     Wright             qhowell@example.net Artist
      9       John   Martinez      belindagilbert@example.net Artist
     10    Candice       Park            sydney78@example.com Artist
     11    Melissa      Yoder           michael77@example.org  Buyer
     12   Kristina      Walsh    hickschristopher@example.org  Buyer
     13    Jessica   Robinson christopherbenjamin@example.com  Buyer
     14       Mark         Wu     

In [ ]:
import pandas as pd

df_users = pd.read_sql_query('SELECT * FROM User', conn)
df_users

,UserID,FirstName,LastName,Email,Role
0,1,Mark,Smith,alison69@example.net,Artist
1,2,Taylor,Andrews,rodriguezmegan@example.com,Artist
2,3,Holly,Alvarado,valentinesean@example.net,Artist
3,4,Jeffrey,Martinez,woodalejandro@example.com,Artist
4,5,Sara,Newman,abright@example.net,Artist
5,6,Brett,Clements,elizabethshaw@example.net,Artist
6,7,Ryan,Harrison,hbridges@example.org,Artist
7,8,Suzanne,Wright,qhowell@example.net,Artist
8,9,John,Martinez,belindagilbert@example.net,Artist
9,10,Candice,Park,sydney78@example.com,Artist


### Design a hash function and apply linear hashing


In [ ]:
import sqlite3

# Function to hash artwork IDs
def hash_artwork_id(artwork_id, num_buckets):
    prefix = artwork_id[:3]
    numeric_part = artwork_id.split('_')[1]

    ascii_sum = ord(prefix[0]) * 1 + ord(prefix[1]) * 2 + ord(prefix[2]) * 3
    for i, digit in enumerate(numeric_part):
        ascii_sum += ord(digit) * (i + 1)

    hash_value = ascii_sum + ord('A') + ord('I') + ord('S') + ord('E')

    final_hash = int(hash_value) % num_buckets

    return final_hash

# Initialize
initial_buckets = 2
bucket_size = 3
num_buckets = initial_buckets
buckets = [[] for _ in range(num_buckets)]
split_pointer = 0  # This is the split pointer, tracking the next bucket to split

def insert(artwork_id):
    global num_buckets, split_pointer

    hash_value = hash_artwork_id(artwork_id, num_buckets)
    bucket = buckets[hash_value]

    # Check if the bucket is full and needs splitting
    if len(bucket) >= bucket_size:
        # Split the bucket pointed by split_pointer
        split_bucket(split_pointer)
        # Recompute hash value after split, as num_buckets may have increased
        hash_value = hash_artwork_id(artwork_id, num_buckets)
        bucket = buckets[hash_value]

    bucket.append(artwork_id)
    print(f"Inserted {artwork_id} into Bucket {hash_value}")
    print(f"Hash value: {hash_artwork_id(artwork_id, num_buckets)}")

def split_bucket(bucket_index):
    global num_buckets, split_pointer

    new_bucket_index = num_buckets
    num_buckets += 1
    buckets.append([])  # Add new empty bucket

    old_bucket = buckets[bucket_index]
    print(f"Splitting Bucket {bucket_index}")

    items_to_rehash = old_bucket[:]
    old_bucket.clear()

    for item in items_to_rehash:
        new_index = hash_artwork_id(item, num_buckets)
        buckets[new_index].append(item)

    # Move the split pointer to the next bucket
    split_pointer = (split_pointer + 1) % initial_buckets

    print_buckets()

def print_buckets():
    print("\nCurrent Bucket States:")
    for i, bucket in enumerate(buckets):
        print(f"Bucket {i}: {bucket}")

# Connect to SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Query to get all ArtworkIDs from the Artwork table
cursor.execute('SELECT ArtworkID FROM Artwork')
artwork_ids = cursor.fetchall()

# Insert artwork IDs into the hash table
for (artwork_id,) in artwork_ids:
    insert(artwork_id)

# Display final bucket states
print("\nFinal Bucket States:")
print_buckets()


Inserted ACR_01 into Bucket 1
Hash value: 1
Inserted ACR_02 into Bucket 1
Hash value: 1
Inserted ACR_03 into Bucket 1
Hash value: 1
Splitting Bucket 0

Current Bucket States:
Bucket 0: []
Bucket 1: ['ACR_01', 'ACR_02', 'ACR_03']
Bucket 2: []
Inserted ACR_04 into Bucket 2
Hash value: 2
Splitting Bucket 1

Current Bucket States:
Bucket 0: []
Bucket 1: ['ACR_01', 'ACR_03']
Bucket 2: ['ACR_04']
Bucket 3: ['ACR_02']
Inserted ACR_05 into Bucket 1
Hash value: 1
Inserted ACR_06 into Bucket 3
Hash value: 3
Splitting Bucket 0

Current Bucket States:
Bucket 0: []
Bucket 1: ['ACR_01', 'ACR_03', 'ACR_05']
Bucket 2: ['ACR_04']
Bucket 3: ['ACR_02', 'ACR_06']
Bucket 4: []
Inserted ACR_07 into Bucket 3
Hash value: 3
Splitting Bucket 1

Current Bucket States:
Bucket 0: []
Bucket 1: ['ACR_05']
Bucket 2: ['ACR_04']
Bucket 3: ['ACR_02', 'ACR_06', 'ACR_07', 'ACR_03']
Bucket 4: []
Bucket 5: ['ACR_01']
Inserted DIG_01 into Bucket 5
Hash value: 5
Inserted DIG_02 into Bucket 1
Hash value: 1
Splitting Bucket 0



In [ ]:
# import sqlite3

# # Function to hash artwork IDs
# def hash_artwork_id(artwork_id, num_buckets):
#     prefix = artwork_id[:3]
#     numeric_part = artwork_id.split('_')[1]

#     ascii_sum = ord(prefix[0]) * 1 + ord(prefix[1]) * 2 + ord(prefix[2]) * 3
#     for i, digit in enumerate(numeric_part):
#         ascii_sum += ord(digit) * (i + 1)

#     #ascii_divisor = ord('A') if 'A' in artwork_id else ord('I')
#     hash_value = ascii_sum + ord('A')+ ord('I')+ ord('S')+ ord('E')

#     final_hash = int(hash_value) % num_buckets

#     return final_hash

# # Initialize
# initial_buckets = 2
# bucket_size = 3
# num_buckets = initial_buckets
# buckets = [[] for _ in range(num_buckets)]
# next_split = 0

# def insert(artwork_id):
#     global num_buckets, next_split

#     hash_value = hash_artwork_id(artwork_id, num_buckets)
#     bucket = buckets[hash_value]

#     if len(bucket) >= bucket_size:
#         split_bucket(hash_value)
#         hash_value = hash_artwork_id(artwork_id, num_buckets)
#         bucket = buckets[hash_value]

#     bucket.append(artwork_id)
#     print(f"Inserted {artwork_id} into Bucket {hash_value}")
#     print(f"Hash value: {hash_artwork_id(artwork_id, num_buckets)}")

# def split_bucket(bucket_index):
#     global num_buckets

#     new_bucket_index = num_buckets
#     num_buckets += 1
#     buckets.append([])  # Add new empty bucket

#     old_bucket = buckets[bucket_index]
#     print(f"Splitting Bucket {bucket_index}")

#     items_to_rehash = old_bucket[:]
#     old_bucket.clear()

#     for item in items_to_rehash:
#         new_index = hash_artwork_id(item, num_buckets)
#         buckets[new_index].append(item)

#     print_buckets()

# def print_buckets():
#     print("\nCurrent Bucket States:")
#     for i, bucket in enumerate(buckets):
#         print(f"Bucket {i}: {bucket}")

# # Connect to SQLite database
# conn = sqlite3.connect('CanvasCart.db')
# cursor = conn.cursor()

# # Query to get all ArtworkIDs from the Artwork table
# cursor.execute('SELECT ArtworkID FROM Artwork')
# artwork_ids = cursor.fetchall()

# # Insert artwork IDs into the hash table
# for (artwork_id,) in artwork_ids:
#     insert(artwork_id)

# # Display final bucket states
# print("\nFinal Bucket States:")
# print_buckets()



### Secondary Index

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Create secondary index on ArtistID
cursor.execute('CREATE INDEX idx_medium ON Artwork (Medium);')
conn.commit()


In [ ]:
cursor.execute('EXPLAIN QUERY PLAN SELECT * FROM Artwork WHERE Medium = "Oil";')
query_plan = cursor.fetchall()

for step in query_plan:
    print(step)


(3, 0, 0, 'SEARCH Artwork USING INDEX idx_medium (Medium=?)')


In [ ]:
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Create secondary index on ArtistID
cursor.execute('CREATE INDEX idx_price ON Artwork (Price);')
conn.commit()

In [ ]:
cursor.execute('EXPLAIN QUERY PLAN SELECT * FROM Artwork WHERE Price <= 1000;')
query_plan = cursor.fetchall()

for step in query_plan:
    print(step)


(3, 0, 0, 'SEARCH Artwork USING INDEX idx_price (Price<?)')


### Clustering Index



In [ ]:
import sqlite3

# Create a non-unique index on ArtworkID
cursor.execute('CREATE INDEX idx_artistID ON Artwork(ArtistID)')

# Commit the changes and close the connection
conn.commit()

In [ ]:
# Use EXPLAIN QUERY PLAN to see how the query planner uses the index
cursor.execute("""
    EXPLAIN QUERY PLAN
    SELECT * FROM Artwork WHERE ArtistID = 1;
""")

# Fetch and print the query plan
query_plan = cursor.fetchall()
for step in query_plan:
    print(step)


(3, 0, 0, 'SEARCH Artwork USING INDEX idx_artistID (ArtistID=?)')


### Query: Information on 5 new contemporary artists

In [ ]:
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Insert new contemporary artists into the User table
artists = [
    ('Banksy', 'Anonymous', 'banksy@example.com', 'Artist'),
    ('Yayoi Kusama', 'Kusama', 'yayoi@example.com', 'Artist'),
    ('Ai Weiwei', 'Weiwei', 'aiweiwei@example.com', 'Artist'),
    ('Damien Hirst', 'Hirst', 'damien@example.com', 'Artist'),
    ('Jeff Koons', 'Koons', 'jeffkoons@example.com', 'Artist')
]

cursor.executemany('''
    INSERT INTO User (FirstName, LastName, Email, Role)
    VALUES (?, ?, ?, ?)
''', artists)

conn.commit()

In [ ]:
import pandas as pd

df_users = pd.read_sql_query('SELECT * FROM User', conn)
df_users

,UserID,FirstName,LastName,Email,Role
0,1,Mark,Smith,alison69@example.net,Artist
1,2,Taylor,Andrews,rodriguezmegan@example.com,Artist
2,3,Holly,Alvarado,valentinesean@example.net,Artist
3,4,Jeffrey,Martinez,woodalejandro@example.com,Artist
4,5,Sara,Newman,abright@example.net,Artist
5,6,Brett,Clements,elizabethshaw@example.net,Artist
6,7,Ryan,Harrison,hbridges@example.org,Artist
7,8,Suzanne,Wright,qhowell@example.net,Artist
8,9,John,Martinez,belindagilbert@example.net,Artist
9,10,Candice,Park,sydney78@example.com,Artist


### adding date column in artworks table

In [ ]:
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Add DateAdded column to the Artwork table
cursor.execute('''
    ALTER TABLE Artwork
    ADD COLUMN DateAdded TEXT;
''')

conn.commit()


In [ ]:
import datetime

fake = Faker()
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Generate a random date between two dates
def generate_random_date(start_date, end_date):
    return fake.date_between(start_date=start_date, end_date=end_date).strftime('%Y-%m-%d')

# Define date range as datetime.date objects
start_date = datetime.date(2021, 8, 1)
end_date = datetime.date(2024, 9, 1)

# Update each Artwork entry with a random DateAdded
cursor.execute('SELECT ArtworkID FROM Artwork')
artwork_ids = cursor.fetchall()

for (artwork_id,) in artwork_ids:
    random_date = generate_random_date(start_date, end_date)
    cursor.execute('''
        UPDATE Artwork
        SET DateAdded = ?
        WHERE ArtworkID = ?
    ''', (random_date, artwork_id))

conn.commit()


In [ ]:
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Query to get all records from the Artwork table
cursor.execute('SELECT * FROM Artwork')
artwork_records = cursor.fetchall()

# Fetch column names
columns = [description[0] for description in cursor.description]

# Create a DataFrame for easier viewing
df = pd.DataFrame(artwork_records, columns=columns)

# Display the DataFrame
print(df)

   ArtworkID  ArtistID                      Title  \
0     OIL_01         1             Than national.   
1     ACR_01         1             It learn baby.   
2     SCU_01         1             Plan age mind.   
3     SCU_02         1               Family best.   
4     ACR_02         2          Memory night six.   
5     DIG_01         2            Call community.   
6     DIG_02         2  Government pull movement.   
7     WAT_01         2              Parent chair.   
8     OIL_02         3           Go like brother.   
9     DIG_03         3        Child deal perhaps.   
10    WAT_02         4            Political grow.   
11    ACR_03         4           Tell name admit.   
12    WAT_03         4                 Carry can.   
13    SCU_03         4             Federal world.   
14    OIL_03         4     Huge never particular.   
15    WAT_04         6        Issue focus minute.   
16    WAT_05         6           Somebody finish.   
17    OIL_04         6          Significant fr

### Query: Extract artwork listings made in the month of August, 2024


In [ ]:
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

query = '''
    SELECT * FROM Artwork
    WHERE DateAdded BETWEEN '2024-08-01' AND '2024-08-31'
'''

cursor.execute(query)
artworks_august_2024 = cursor.fetchall()

# Fetch column names
columns = [description[0] for description in cursor.description]

# Create a DataFrame for easier viewing
df_august_2024 = pd.DataFrame(artworks_august_2024, columns=columns)
df_august_2024

,ArtworkID,ArtistID,Title,Description,ImageURL,Medium,Style,Price,Availability,DateAdded
0,ACR_05,7,Yard stand sit.,Better fine indicate according. Mr there struc...,http://nguyen.com/,Acrylic,Realism,9500.44,1,2024-08-19
1,OIL_05,7,Cell soon.,Smile build forget watch city memory which. Mo...,http://www.moody.com/,Oil,Abstract,743.33,14,2024-08-03


### adding time column in order table

In [ ]:

# Define the SQL query to select all rows from the Orders table
query = 'SELECT * FROM Orders'

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the column names
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 80)

# Print the results
for row in results:
    print(f"{' | '.join(map(str, row))}")

OrderID | BuyerID | OrderDate | TotalAmount | PaymentStatus | ShippingStatus | OrderTime
--------------------------------------------------------------------------------
1 | 11 | 2024-03-20 | 1692.68 | Failed | Order Confirmed | 15:35:00
2 | 11 | 2023-10-18 | 3162.51 | Failed | Canceled | 14:22:00
3 | 11 | 2023-05-12 | 272.2 | Failed | Order Confirmed | 21:18:00
4 | 12 | 2023-03-13 | 4965.8 | Completed | Canceled | 19:30:00
5 | 13 | 2023-05-03 | 4014.39 | Completed | Canceled | 14:33:00
6 | 13 | 2023-11-22 | 3225.23 | Completed | Order Confirmed | 19:05:00
7 | 13 | 2022-09-17 | 2838.7 | Completed | Delivered | 18:05:00
8 | 15 | 2022-08-01 | 4904.53 | Failed | Order Confirmed | 13:29:00
9 | 16 | 2021-07-07 | 2948.39 | Completed | Shipped | 20:27:00
10 | 17 | 2024-07-06 | 2766.17 | Pending | Shipped | 18:23:00
11 | 18 | 2021-03-02 | 2613.22 | Failed | Shipped | 13:08:00
12 | 18 | 2022-03-01 | 717.59 | Completed | Canceled | 15:51:00
13 | 18 | 2023-01-08 | 1051.79 | Failed | Order Confirm

In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()

# Query to select all columns from the Orders table
cursor.execute('SELECT * FROM Orders')
rows = cursor.fetchall()

# Convert the data into a DataFrame for better readability
df_orders = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
df_orders

,OrderID,BuyerID,OrderDate,TotalAmount,PaymentStatus,ShippingStatus,OrderTime
0,1,11,2024-03-20,1692.68,Failed,Order Confirmed,15:35:00
1,2,11,2023-10-18,3162.51,Failed,Canceled,14:22:00
2,3,11,2023-05-12,272.20,Failed,Order Confirmed,21:18:00
3,4,12,2023-03-13,4965.80,Completed,Canceled,19:30:00
4,5,13,2023-05-03,4014.39,Completed,Canceled,14:33:00
...,...,...,...,...,...,...,...
60,61,46,2022-06-15,3638.78,Completed,Canceled,22:23:00
61,62,46,2022-12-26,1669.33,Pending,Delivered,22:27:00
62,63,48,2021-01-15,1159.73,Failed,Canceled,17:42:00
63,64,49,2024-05-26,4148.04,Completed,Canceled,21:53:00


### Query: Remove all artwork purchases made after 7PM on August 15, 2024

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('CanvasCart.db')
cursor = conn.cursor()


cutoff_date = '2024-08-15'
cutoff_time = '19:00:00'

cutoff_datetime = f'{cutoff_date} {cutoff_time}'

cursor.execute('''
    DELETE FROM Orders
    WHERE OrderDate = ? AND OrderTime > ?
''', (cutoff_date, cutoff_time))

# Commit the changes and close the connection
conn.commit()

# Check how many rows were affected
print(f"Rows deleted: {cursor.rowcount}")



Rows deleted: 0


## LAB2

Initial SQL queries

In [ ]:
import time

# Function to execute and time the query
def execute_and_time_query(query):
    start_time = time.time()  # Start timer
    cursor.execute(query)     # Execute query
    results = cursor.fetchall()  # Fetch results
    end_time = time.time()    # End timer
    return results, end_time - start_time


Query 1: Extract a list of all artist_name, artwork who have artwork listings in all months in 2023.


Initial

In [ ]:
import sqlite3

# Define the SQL query
query = '''
    SELECT u.FirstName AS ArtistFirstName, u.LastName AS ArtistLastName, art.Title AS ArtworkTitle
    FROM User u,  Artwork art
    WHERE u.UserID = art.ArtistID AND strftime('%Y', art.DateAdded) = '2023'
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Display the results
for row in results:
    artist_first_name = row[0]
    artist_last_name = row[1]
    artwork_title = row[2]
    print(f"Artist: {artist_first_name} {artist_last_name}, Artwork: {artwork_title}")

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Artist: Mark Smith, Artwork: It learn baby.
Artist: Mark Smith, Artwork: Plan age mind.
Artist: Mark Smith, Artwork: Family best.
Artist: Taylor Andrews, Artwork: Memory night six.
Artist: Taylor Andrews, Artwork: Government pull movement.
Artist: Taylor Andrews, Artwork: Parent chair.
Artist: Jeffrey Martinez, Artwork: Carry can.
Artist: Jeffrey Martinez, Artwork: Federal world.
Artist: Jeffrey Martinez, Artwork: Huge never particular.
Artist: Brett Clements, Artwork: Present respond fast.
Artist: Suzanne Wright, Artwork: Fact measure.
Artist: John Martinez, Artwork: Bad dream president.
Artist: John Martinez, Artwork: Agency before.
Artist: Candice Park, Artwork: Partner.
Execution time for non-optimized query: 0.000153 seconds


Optimised

In [ ]:
import sqlite3

# Define the SQL query
query = '''
  SELECT u.FirstName, u.LastName, art.Title
  FROM (
      SELECT u.FirstName, u.LastName, u.UserID
      FROM User u
  ) u
  JOIN (
      SELECT art.ArtistID, art.Title
      FROM Artwork art
      WHERE strftime('%Y', art.DateAdded) = '2023'
  ) art
  ON u.UserID = art.ArtistID;

'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Display the results
for row in results:
    artist_first_name = row[0]
    artist_last_name = row[1]
    artwork_title = row[2]
    print(f"Artist: {artist_first_name} {artist_last_name}, Artwork: {artwork_title}")

# Measure time for the second (optimized) query
results_2, time_2 = execute_and_time_query(query)
print(f"Execution time for optimized query: {time_2:.6f} seconds")

Artist: Mark Smith, Artwork: It learn baby.
Artist: Mark Smith, Artwork: Plan age mind.
Artist: Mark Smith, Artwork: Family best.
Artist: Taylor Andrews, Artwork: Memory night six.
Artist: Taylor Andrews, Artwork: Government pull movement.
Artist: Taylor Andrews, Artwork: Parent chair.
Artist: Jeffrey Martinez, Artwork: Carry can.
Artist: Jeffrey Martinez, Artwork: Federal world.
Artist: Jeffrey Martinez, Artwork: Huge never particular.
Artist: Brett Clements, Artwork: Present respond fast.
Artist: Suzanne Wright, Artwork: Fact measure.
Artist: John Martinez, Artwork: Bad dream president.
Artist: John Martinez, Artwork: Agency before.
Artist: Candice Park, Artwork: Partner.
Execution time for optimized query: 0.000170 seconds


Query 2: Get all artists who have sculptures

Initial

In [ ]:
import sqlite3

query = '''
    SELECT u.FirstName, u.LastName
    FROM User u, Artwork art
    WHERE u.UserID = art.ArtistID AND strftime('%Y', art.DateAdded) = '2023' AND art.medium = 'Sculpture'
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the names of artists with at least one sculpture
print("Artists with at least one sculpture:")
for row in results:
    artist_first_name = row[0]
    artist_last_name = row[1]
    print(f"{artist_first_name} {artist_last_name}")

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Artists with at least one sculpture:
Mark Smith
Mark Smith
Jeffrey Martinez
Candice Park
Execution time for non-optimized query: 0.000752 seconds


Optimised

In [ ]:
import sqlite3

query = '''
SELECT u.FirstName, u.LastName
      FROM (SELECT u.FirstName, u.LastName, u.UserID
            FROM User u) u
JOIN (SELECT art.ArtistID, art.Title
      FROM Artwork art
      WHERE strftime('%Y', art.DateAdded) = '2023' AND art.Medium = 'Sculpture') art
ON u.UserID = art.ArtistID;
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the names of artists with at least one sculpture
print("Artists with at least one sculpture:")
for row in results:
    artist_first_name = row[0]
    artist_last_name = row[1]
    print(f"{artist_first_name} {artist_last_name}")

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")


Artists with at least one sculpture:
Mark Smith
Mark Smith
Jeffrey Martinez
Candice Park
Execution time for non-optimized query: 0.000107 seconds


Query 3: Get artists who do not have any artwork listings

Initial

In [ ]:
from tabulate import tabulate

query = '''
    SELECT u.UserID, u.FirstName, u.LastName, a.Biography, a.PortfolioURL
    FROM User u
    JOIN Artist a ON u.UserID = a.ArtistID
    LEFT JOIN Artwork art ON art.ArtistID = a.ArtistID
    WHERE art.ArtworkID IS NULL
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the artist profiles with no artwork listings
table_headers = ["UserID", "FirstName", "LastName", "Biography", "Portfolio URL"]
table_data = [list(row) for row in results]

# Print the table
print("Artists with no artwork listings:")
print(tabulate(table_data, headers=table_headers, tablefmt='grid'))

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Artists with no artwork listings:
+----------+-------------+------------+----------------------------------------------------------------------------+----------------------------+
|   UserID | FirstName   | LastName   | Biography                                                                  | Portfolio URL              |
+==========+=============+============+============================================================================+============================+
|        5 | Sara        | Newman     | Mrs paper art first grow. Accept tax person quickly activity subject meet. | http://wagner-jordan.info/ |
|          |             |            | There fight down sense against five.                                       |                            |
|          |             |            | View main sea whose. Camera task put any arm tough.                        |                            |
|          |             |            | Understand yourself customer call.                

Optimised

In [ ]:
from tabulate import tabulate

query = '''
    SELECT u.UserID, u.FirstName, u.LastName, a_wo_art.Biography, a_wo_art.PortfolioURL
    FROM (
        SELECT*
    FROM Artist a
    LEFT JOIN (
                SELECT art.ArtistID, art.ArtworkID
                From Artwork art
            )art
    ON art.ArtistID = a.ArtistID
    WHERE art.ArtworkID IS NULL ) AS a_wo_ART
    JOIN (
        SELECT u.UserID, u.FirstName, u.LastName
        FROM User u
    ) u
    ON u.UserID = a_wo_art.ArtistID;'''
# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the artist profiles with no artwork listings
table_headers = ["UserID", "FirstName", "LastName", "Biography", "Portfolio URL"]
table_data = [list(row) for row in results]

# Print the table
print("Artists with no artwork listings:")
print(tabulate(table_data, headers=table_headers, tablefmt='grid'))

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Artists with no artwork listings:
+----------+-------------+------------+----------------------------------------------------------------------------+----------------------------+
|   UserID | FirstName   | LastName   | Biography                                                                  | Portfolio URL              |
+==========+=============+============+============================================================================+============================+
|        5 | Sara        | Newman     | Mrs paper art first grow. Accept tax person quickly activity subject meet. | http://wagner-jordan.info/ |
|          |             |            | There fight down sense against five.                                       |                            |
|          |             |            | View main sea whose. Camera task put any arm tough.                        |                            |
|          |             |            | Understand yourself customer call.                

Query 4:  get buyers who purchased oil paintings in 2022

Initial

In [ ]:
query = '''
    SELECT DISTINCT u.UserID, u.FirstName, u.LastName
    FROM Orders o,Item i, Artwork art, User u
    WHERE u.UserID = o.BuyerID AND o.OrderID = i.OrderID AND i.ArtworkID = art.ArtworkID AND art.Medium = 'Oil' AND strftime('%Y', o.OrderDate) = '2022'
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the list of buyers
print("Buyers who purchased oil paintings in 2022:")
for row in results:
    buyer_id = row[0]
    first_name = row[1]
    last_name = row[2]
    print(f"BuyerID: {buyer_id}, Name: {first_name} {last_name}")

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Buyers who purchased oil paintings in 2022:
BuyerID: 20, Name: Robert Roberts
BuyerID: 13, Name: Jessica Robinson
BuyerID: 33, Name: Candace Bowman
BuyerID: 46, Name: David Price
BuyerID: 40, Name: Melissa Crawford
BuyerID: 15, Name: Eugene Rhodes
BuyerID: 37, Name: Adam Daniel
BuyerID: 41, Name: Russell Wiley
Execution time for non-optimized query: 0.000207 seconds


Optimised

In [ ]:
query = '''
    SELECT DISTINCT
        u.UserID,
        u.FirstName,
        u.LastName
    FROM
        (SELECT i.OrderID, i.ArtworkID
        FROM Item i) i
    JOIN
        (SELECT art.ArtworkID
        FROM Artwork art
        WHERE art.Medium = 'Oil') art
    ON i.ArtworkID = art.ArtworkID
    JOIN
        (SELECT o.OrderID, o.BuyerID
        FROM Orders o
        WHERE strftime('%Y', o.OrderDate) = '2022') o
    ON i.OrderID = o.OrderID
    JOIN
        (SELECT  u.UserID, u.FirstName, u.LastName
        FROM User u) u
    ON o.BuyerID = u.UserID;
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print the list of buyers
print("Buyers who purchased oil paintings in 2022:")
for row in results:
    buyer_id = row[0]
    first_name = row[1]
    last_name = row[2]
    print(f"BuyerID: {buyer_id}, Name: {first_name} {last_name}")

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Buyers who purchased oil paintings in 2022:
BuyerID: 20, Name: Robert Roberts
BuyerID: 13, Name: Jessica Robinson
BuyerID: 33, Name: Candace Bowman
BuyerID: 46, Name: David Price
BuyerID: 40, Name: Melissa Crawford
BuyerID: 15, Name: Eugene Rhodes
BuyerID: 37, Name: Adam Daniel
BuyerID: 41, Name: Russell Wiley
Execution time for non-optimized query: 0.000155 seconds


query 5 : From the above list, derive a list of the artists and their profile information.


Initial

In [ ]:
from tabulate import tabulate

query = '''
    SELECT DISTINCT u.FirstName AS ArtistFirstName, u.LastName AS ArtistLastName, art.Biography, art.PortfolioURL
    FROM User u, Artist art, Artwork a, Item i, Orders o
    WHERE u.UserID = art.ArtistID
    AND art.ArtistID = a.ArtistID
    AND a.ArtworkID = i.ArtworkID
    AND i.OrderID = o.OrderID
    AND a.Medium = 'Oil'
    AND strftime('%Y', o.OrderDate) = '2022'
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Define headers for the table
headers = ["Artist First Name", "Artist Last Name", "Biography", "Portfolio URL"]

# Print the results in tabular format
print(tabulate(results, headers=headers, tablefmt="pretty"))

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

+-------------------+------------------+----------------------------------------------------------------------------------------------------------------------+-------------------------------+
| Artist First Name | Artist Last Name |                                                      Biography                                                       |         Portfolio URL         |
+-------------------+------------------+----------------------------------------------------------------------------------------------------------------------+-------------------------------+
|       Mark        |      Smith       |   Partner expert discover hold appear. Value offer practice want. Least forget suffer final call PM leave writer.    |    https://www.chung.com/     |
|       Holly       |     Alvarado     |                              Quality mind deep through. Law mean term spring something.                              | https://www.hall-bentley.biz/ |
|                   |                  |

Optimised

In [ ]:
from tabulate import tabulate

query = '''
    SELECT DISTINCT
    u.UserID,
    u.FirstName,
    u.LastName,
    a.biography,
    a.portfolioURL
FROM
    (SELECT i.OrderID, i.ArtworkID
    FROM Item i) AS i
JOIN
    (SELECT art.ArtworkID, art.ArtistID
    FROM Artwork art
    WHERE art.Medium = 'Oil') AS art
ON i.ArtworkID = art.ArtworkID
JOIN
    (SELECT o.OrderID, o.BuyerID
    FROM Orders o
    WHERE strftime('%Y', o.OrderDate) = '2022') AS o
ON i.OrderID = o.OrderID
JOIN
    (SELECT a.ArtistID, a.biography, a.portfolioURL
    FROM Artist a) AS a
ON art.ArtistID = a.ArtistID
JOIN
    (SELECT u.UserID, u.FirstName, u.LastName
    FROM User u) AS u
ON a.ArtistID = u.UserID;


'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Define headers for the table
headers = ["Artist First Name", "Artist Last Name", "Biography", "Portfolio URL"]

# Print the results in tabular format
print(tabulate(results, headers=headers, tablefmt="pretty"))

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

+----+-------------------+------------------+----------------------------------------------------------------------------------------------------------------------+-------------------------------+
|    | Artist First Name | Artist Last Name |                                                      Biography                                                       |         Portfolio URL         |
+----+-------------------+------------------+----------------------------------------------------------------------------------------------------------------------+-------------------------------+
| 1  |       Mark        |      Smith       |   Partner expert discover hold appear. Value offer practice want. Least forget suffer final call PM leave writer.    |    https://www.chung.com/     |
| 3  |       Holly       |     Alvarado     |                              Quality mind deep through. Law mean term spring something.                              | https://www.hall-bentley.biz/ |
|    |         

query 6 : Derive a list of all buyer_profiles who have not made any purchases.

Initial

In [ ]:
from tabulate import tabulate

# Define the SQL query to get buyers who have not made any purchases
query = '''
    SELECT b.BuyerID, u.FirstName, u.LastName, b.ShippingAddress, b.Membership
    FROM Buyer b
    JOIN User u ON b.BuyerID = u.UserID
    LEFT JOIN Orders o ON b.BuyerID = o.BuyerID
    WHERE o.OrderID IS NULL
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Prepare data for tabulate
table_headers = ["BuyerID", "FirstName", "LastName", "ShippingAddress", "Membership"]
table_data = [list(row) for row in results]

# Print the table
print("Buyers who have not made any purchases:")
print(tabulate(table_data, headers=table_headers, tablefmt='grid'))

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Buyers who have not made any purchases:
+-----------+-------------+------------+-------------------------------+--------------+
|   BuyerID | FirstName   | LastName   | ShippingAddress               |   Membership |
+===========+=============+============+===============================+==============+
|        14 | Mark        | Wu         | 269 Smith Trail Apt. 772      |            1 |
|           |             |            | West Ashleyville, PW 68327    |              |
+-----------+-------------+------------+-------------------------------+--------------+
|        24 | Jacqueline  | Henson     | 279 Megan Fields              |            1 |
|           |             |            | Warrenmouth, RI 02451         |              |
+-----------+-------------+------------+-------------------------------+--------------+
|        28 | Pamela      | Harris     | 9805 Jessica Greens Suite 649 |            1 |
|           |             |            | New Williamside, AK 47828     |        

Optimised

In [ ]:
from tabulate import tabulate

# Define the SQL query to get buyers who have not made any purchases
query = '''
    SELECT  b_wo_o.BuyerID, u.FirstName, u.LastName,  b_wo_o.ShippingAddress,  b_wo_o.Membership
    FROM (
        SELECT *
        FROM Buyer b
        LEFT JOIN (SELECT o.OrderID, o.BuyerID
                   FROM Orders o
                ) o
        ON b.BuyerID=o.BuyerID
        WHERE o.OrderID IS NULL) AS b_wo_o
    JOIN(
        SELECT u.UserID, u.FirstName, u.LastName
        FROM User u
    ) u
    ON  b_wo_o.BuyerID = u.UserID;
'''
# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Prepare data for tabulate
table_headers = ["BuyerID", "FirstName", "LastName", "ShippingAddress", "Membership"]
table_data = [list(row) for row in results]

# Print the table
print("Buyers who have not made any purchases:")
print(tabulate(table_data, headers=table_headers, tablefmt='grid'))

# Measure time for the first (non-optimized) query
results_1, time_1 = execute_and_time_query(query)
print(f"Execution time for non-optimized query: {time_1:.6f} seconds")

Buyers who have not made any purchases:
+-----------+-------------+------------+-------------------------------+--------------+
|   BuyerID | FirstName   | LastName   | ShippingAddress               |   Membership |
+===========+=============+============+===============================+==============+
|        14 | Mark        | Wu         | 269 Smith Trail Apt. 772      |            1 |
|           |             |            | West Ashleyville, PW 68327    |              |
+-----------+-------------+------------+-------------------------------+--------------+
|        24 | Jacqueline  | Henson     | 279 Megan Fields              |            1 |
|           |             |            | Warrenmouth, RI 02451         |              |
+-----------+-------------+------------+-------------------------------+--------------+
|        28 | Pamela      | Harris     | 9805 Jessica Greens Suite 649 |            1 |
|           |             |            | New Williamside, AK 47828     |        